# Football Video Analyst - Test Notebook

This notebook demonstrates how to use the Football Video Analyst modules to analyze football highlight videos using Gemini 2.5 Pro.

## Setup API Key

First, set up your Google API key. You can either get it from environment variables or set it directly.

In [ ]:
# For Google Colab
# from google.colab import userdata
# GOOGLE_API_KEY = userdata.get('GOOGLE_AI_STUDIO')

# For local environment
import os
from dotenv import load_dotenv

load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

# Verify API key exists
if GOOGLE_API_KEY is None:
    print("⚠️ API key not found. Please set GOOGLE_API_KEY in .env file.")
else:
    print("✅ API key loaded successfully!")

## Import Required Libraries

Import the Google Generative AI library and our custom modules.

In [ ]:
import os
import time
import json
from google import genai
from google.genai import types

# Import our custom modules
import sys
sys.path.append('.')
from modules.video_utils import extract_youtube_id, embed_youtube_video
from modules.analysis import analyze_football_video, get_rating_class

## Initialize Gemini Client

Create a client instance with your API key.

In [ ]:
# Initialize the Gemini client
client = genai.Client(api_key=GOOGLE_API_KEY)

# Set the model name
MODEL_NAME = "gemini-2.5-pro-exp-03-25"

## Analyze a Football Video

Enter a YouTube URL of a football highlights video to analyze.

In [ ]:
# Example football highlight video URL
# Manchester City vs Real Madrid UCL 2023 semi-final
youtube_url = "https://www.youtube.com/watch?v=LMhe2egLsrQ"

# Extract video ID for embedding
video_id = extract_youtube_id(youtube_url)
print(f"Video ID: {video_id}")

# Display embedded video
from IPython.display import HTML
HTML(embed_youtube_video(youtube_url))

## Run Analysis

Analyze the football video using Gemini 2.5 Pro.

In [ ]:
# Measure execution time
start_time = time.time()

# Run the analysis
print("Analyzing football video with Gemini 2.5 Pro...")
analysis_result = analyze_football_video(youtube_url, client, MODEL_NAME)

# Calculate elapsed time
elapsed_time = time.time() - start_time
print(f"Analysis completed in {elapsed_time:.2f} seconds")

# Check if we got an error
if "error" in analysis_result:
    print(f"Error: {analysis_result['error']}")
    if "raw_response" in analysis_result:
        print("Raw response:")
        print(analysis_result["raw_response"])

## Display Match Information

Show the match details and score.

In [ ]:
# Extract match information
if "error" not in analysis_result:
    competition = analysis_result.get("competition", "Unknown Competition")
    match_date = analysis_result.get("match_date", "")
    match_stage = analysis_result.get("match_stage", "")
    team1 = analysis_result.get("team1", {})
    team2 = analysis_result.get("team2", {})
    
    team1_name = team1.get("name", "Team 1")
    team2_name = team2.get("name", "Team 2")
    team1_score = team1.get("score", "?")
    team2_score = team2.get("score", "?")
    
    print(f"{competition} {match_stage} {match_date}")
    print(f"{team1_name} {team1_score} - {team2_score} {team2_name}")
    
    print("\nMatch Summary:")
    print(analysis_result.get("match_summary", "No summary available"))

## Display Player Ratings and Analysis

Show the player ratings and performance analysis for both teams.

In [ ]:
# Function to display player ratings and analysis
def display_team_analysis(team_data):
    team_name = team_data.get("name", "Unknown Team")
    print(f"\n{team_name} Player Ratings:")
    print("-" * 50)
    
    for player in team_data.get("players", []):
        name = player.get("name", "Unknown Player")
        rating = player.get("rating", "?")
        analysis = player.get("analysis", "No analysis available")
        
        print(f"{name}: {rating}/10")
        print(f"Analysis: {analysis}")
        
        # Display key moments if available
        key_moments = player.get("key_moments", [])
        if key_moments:
            print("Key moments:")
            for moment in key_moments:
                time = moment.get("time", "?")
                desc = moment.get("description", "")
                print(f"  • {time}: {desc}")
        print("-" * 50)

# Display analysis for both teams
if "error" not in analysis_result:
    # Team 1 analysis
    display_team_analysis(team1)
    
    # Team 2 analysis
    display_team_analysis(team2)

## Save Analysis Results

Save the analysis results to a JSON file for future reference.

In [ ]:
from google.genai import types

youtube_url = "https://www.youtube.com/watch?v=LMhe2egLsrQ"

prompt = """Analyze the following YouTube video content. Provide a detailed breakdown:

1. **Main Thesis/Claim:** What is the central point the creator is making?
2. **Key Topics:** List the main subjects discussed, referencing specific sections or timestamps if possible.
3. **Call to Action:** Identify any explicit requests made to the viewers.
4. **Summary:** Provide a concise summary of the video content.

Use the provided title, chapter timestamps/descriptions, and description text if available.
"""

# Analyze the video
response = client.models.generate_content(
    model="gemini-2.5-pro-exp-03-25",
    contents=types.Content(
        parts=[
            types.Part(text=prompt),
            types.Part(
                file_data=types.FileData(file_uri=youtube_url)
            )
        ]
    )
)

print(response.text)
